In [ ]:
from imports import *

In [ ]:
datadir = "data/Ge71/ExtraEnergy/"
decayOnly = datadir+"Ge71-vacuum_00250531_000000.root"
singleZip = datadir+"Ge71-iZIP7_00250531_000000.root"
centerCUTE = datadir+"Ge71-centerCUTE_23250531_000000.root"
contamCUTE = datadir+"Ge71-contamCUTE_23250531_000000.root"

In [ ]:
Cf252n1 = datadir+"Ge-1n_00250601_000000.root"
Cf252n337 = datadir+"Ge-3.37n_00250601_000000.root"

In [ ]:
def Ge71energy(file,subtitle,name):
    """Processes job with direct Ge-71 isotope source; every event is a decay."""
    branches = ['EventNum','KE']
    mcDecays = CDataFrame("G4SimDir/mcDecays", file).AsNumpy(branches)
    KEsum = np.bincount(mcDecays['EventNum'].astype(int), weights=mcDecays['KE']/1e3)
    plt.hist(KEsum,bins=100)
    plt.title("Ge-71 decay X-rays: "+subtitle)
    plt.xlabel("Visible decay energy [keV]")
    plt.savefig(datadir+f"Ge71_{name}.png")

In [ ]:
def Ge71daughters(file,subtitle,name):
    """Processes neutron activation job, looks for daughters from a Ge-71 decay
       mixed in with many others.  Uses parent of Ga-71 nucleus as selector."""
    branches = ['EventNum','Parent','PType','PName','KE']
    mcDecays = CDataFrame("G4SimDir/mcDecays", file).AsNumpy(branches)

    evtInt = mcDecays['EventNum'].astype(int)
    parInt = mcDecays['Parent'].astype(int)
    evPar = np.transpose([evtInt,parInt])
    isGe71 = (mcDecays['PType']==71031)    # Exactly one per Ge-71 decay
    geEP = np.unique(evPar[isGe71],axis=1)

    # KE of Ge-71 daughters in all events (non-Ge71 are zero)
    secKE = mcDecays['KE']*np.isin(evPar,geEP).all(-1)

    # Use Numpy to sum KE by event and parent as a histogram
    evtBins = int(evtInt.max()+1)
    parBins = int(parInt.max()+1)
    KEsum,_,_ = np.histogram2d(evtInt,parInt,bins=[evtBins,parBins],weights=secKE)
    
    plt.hist(KEsum[KEsum>0.],bins=100)
    plt.title("Ge-71 decay X-rays: "+subtitle)
    plt.xlabel("Visible decay energy [keV]")
    plt.savefig(datadir+f"Ge71_{name}.png")

In [ ]:
branches = ['EventNum','Parent','PType','PName','KE']
mcDecays = CDataFrame("G4SimDir/mcDecays", Cf252n1).AsNumpy(branches)
evtInt = mcDecays['EventNum'].astype(int)
parInt = mcDecays['Parent'].astype(int)
evPar = np.transpose([evtInt,parInt])
isGe71 = (mcDecays['PType']==71031)    # Exactly one per Ge-71 decay
geEP = np.unique(evPar[isGe71],axis=1)

In [ ]:
geEP

In [ ]:
KEsum = np.array([mcDecays['KE'][np.isin(evPar,ge).all(-1)].shape[0] for ge in geEP])

In [ ]:
plt.hist(KEsum)

In [ ]:
Ge71daughters(decayOnly,"Decay in vacuum","vacuum")

In [ ]:
Ge71daughters(Cf252n1,"Neutron activation","Cf252n1")

In [ ]:
Ge71energy(decayOnly,"Decay in vacuum","vacuum")

In [ ]:
Ge71energy(singleZip,"Decay in iZIP7","iZIP7")

In [ ]:
Ge71energy(centerCUTE,"At (0,0,0) in CUTE","centerCUTE")

In [ ]:
Ge71energy(contamCUTE,"in all CUTE HV100mm","contamCUTE")